In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


# Part 1

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(source, 'html5lib')

In [4]:
table1 = soup.find('table',{'class':'wikitable sortable'})
data1 = table1.find_all('td')

In [5]:
counter = 0
table = []
row = []
for details in data1:
    if counter < 3:
        row.append(details.get_text().strip())
        counter+=1
    else:
        table.append(row)
        counter = 0
        row = []
        row.append(details.get_text().strip())
        counter+=1
headers=['Postcode','Borough','Neighbourhood']
df = pd.DataFrame(table, columns=headers)
def cleaning (row):
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
    return row
df1 = df.apply(cleaning, axis=1)
df1 = df1[df1['Borough'] != 'Not assigned']

In [6]:
df2 = df1.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
df2.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
df2.shape

(103, 3)

# Part 2

In [8]:
pip install geocoder

    100% |████████████████████████████████| 102kB 19.4MB/s 
Note: you may need to restart the kernel to use updated packages.


In [9]:
import geocoder # import geocoder

def get_coordinates1 (row):
    # initialize your variable to None
    lat_lng_coords = None
    print(row['Postcode'])
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(row['Postcode']))
        lat_lng_coords = g.latlng
        print('1')

    row['Latitude'] = lat_lng_coords[0]
    row['Longitude'] = lat_lng_coords[1]
    print(row['Latitude'], row['Longitude'])
    return row

coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
def get_coordinates2 (row):
    temperature_df = coordinates_df[coordinates_df['Postal Code'] == row['Postcode']]
    row['Latitude'] = float(temperature_df['Latitude'])
    row['Longitude'] = float(temperature_df['Longitude'])
    return row
    
df_with_coordinates = df2
df_with_coordinates['Latitude'] = np.nan
df_with_coordinates['Longitude'] = np.nan
df_with_coordinates = df_with_coordinates.apply(get_coordinates2, axis=1)
df_with_coordinates.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Part 3

## Cluster the postcodes having same borough 

In [38]:
import folium
# create map of New York using latitude and longitude values
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
df_toronto_neighborhoods = df_with_coordinates.rename(index=str, columns={"Neighbourhood": "Neighborhood"})
df_toronto_neighborhoods['Neighborhood'] = '['+ df_toronto_neighborhoods['Postcode'] + '] ' + df_toronto_neighborhoods['Neighborhood']
# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_neighborhoods['Latitude'], df_toronto_neighborhoods['Longitude'], df_toronto_neighborhoods['Borough'], df_toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough.upper())
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [39]:
df_toronto_neighborhoods.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[M1B] Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"[M1C] Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"[M1E] Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,[M1G] Woburn,43.770992,-79.216917
4,M1H,Scarborough,[M1H] Cedarbrae,43.773136,-79.239476


In [40]:
CLIENT_ID = '24N2AQAZGNSVEI3AUA54V1L1ICDMTVJTVW3LOW5LPVTLF5PL' #'client-ID' Foursquare ID
CLIENT_SECRET = 'TSHJO0PF3JH1KR5C2DCJFT0CNC5OSBSRAUFND3JHQQXFETJR' #  Foursquare Secret
VERSION = '20181231'
print('My Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My Credentails:
CLIENT_ID: 24N2AQAZGNSVEI3AUA54V1L1ICDMTVJTVW3LOW5LPVTLF5PL
CLIENT_SECRET:TSHJO0PF3JH1KR5C2DCJFT0CNC5OSBSRAUFND3JHQQXFETJR


In [41]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
df_toronto_venues = getNearbyVenues(names=df_toronto_neighborhoods['Neighborhood'],
                                   latitudes=df_toronto_neighborhoods['Latitude'],
                                   longitudes=df_toronto_neighborhoods['Longitude'])
print(df_toronto_venues.shape)
df_toronto_venues.groupby('Neighborhood').count()

[M1B] Rouge, Malvern
[M1C] Highland Creek, Rouge Hill, Port Union
[M1E] Guildwood, Morningside, West Hill
[M1G] Woburn
[M1H] Cedarbrae
[M1J] Scarborough Village
[M1K] East Birchmount Park, Ionview, Kennedy Park
[M1L] Clairlea, Golden Mile, Oakridge
[M1M] Cliffcrest, Cliffside, Scarborough Village West
[M1N] Birch Cliff, Cliffside West
[M1P] Dorset Park, Scarborough Town Centre, Wexford Heights
[M1R] Maryvale, Wexford
[M1S] Agincourt
[M1T] Clarks Corners, Sullivan, Tam O'Shanter
[M1V] Agincourt North, L'Amoreaux East, Milliken, Steeles East
[M1W] L'Amoreaux West
[M1X] Upper Rouge
[M2H] Hillcrest Village
[M2J] Fairview, Henry Farm, Oriole
[M2K] Bayview Village
[M2L] Silver Hills, York Mills
[M2M] Newtonbrook, Willowdale
[M2N] Willowdale South
[M2P] York Mills West
[M2R] Willowdale West
[M3A] Parkwoods
[M3B] Don Mills North
[M3C] Flemingdon Park, Don Mills South
[M3H] Bathurst Manor, Downsview North, Wilson Heights
[M3J] Northwood Park, York University
[M3K] CFB Toronto, Downsview East
[M

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"[M1B] Rouge, Malvern",2,2,2,2,2,2
"[M1C] Highland Creek, Rouge Hill, Port Union",1,1,1,1,1,1
"[M1E] Guildwood, Morningside, West Hill",8,8,8,8,8,8
[M1G] Woburn,3,3,3,3,3,3
[M1H] Cedarbrae,7,7,7,7,7,7
[M1J] Scarborough Village,2,2,2,2,2,2
"[M1K] East Birchmount Park, Ionview, Kennedy Park",5,5,5,5,5,5
"[M1L] Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"[M1M] Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2


In [43]:
print('There are {} uniques categories.'.format(len(df_toronto_venues['Venue Category'].unique())))

There are 280 uniques categories.


In [44]:
# one hot encoding
toronto_onehot = pd.get_dummies(df_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = df_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
#toronto_onehot.shape

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"[M1B] Rouge, Malvern",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[M1C] Highland Creek, Rouge Hill, Port Union",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[M1E] Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,[M1G] Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,[M1H] Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Top 5 Types of Venues per Postal Code

In [48]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----[M1B] Rouge, Malvern----
                             venue  freq
0             Fast Food Restaurant   0.5
1                       Print Shop   0.5
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----[M1C] Highland Creek, Rouge Hill, Port Union----
                             venue  freq
0                              Bar   1.0
1               Mexican Restaurant   0.0
2              Monument / Landmark   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----[M1E] Guildwood, Morningside, West Hill----
                 venue  freq
0                  Spa  0.12
1  Rental Car Location  0.12
2       Medical Center  0.12
3    Electronics Store  0.12
4       Breakfast Spot  0.12


----[M1G] Woburn----
                 venue  freq
0          Coffee Shop  0.67
1    Korean Restaurant  0.33
2          Yoga Studio  0.00
3                Motel  0.00
4  Monument / Landmark  0.00


-

## Most common Venues for each postal code

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"[M1B] Rouge, Malvern",Print Shop,Fast Food Restaurant,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
1,"[M1C] Highland Creek, Rouge Hill, Port Union",Bar,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,"[M1E] Guildwood, Morningside, West Hill",Electronics Store,Spa,Rental Car Location,Mexican Restaurant,Medical Center,Breakfast Spot,Pizza Place,Intersection,Donut Shop,Doner Restaurant
3,[M1G] Woburn,Coffee Shop,Korean Restaurant,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
4,[M1H] Cedarbrae,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Thai Restaurant,Bakery,Bank,Athletics & Sports,Department Store,Dessert Shop,Dim Sum Restaurant


## Clustering

In [51]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([5, 4, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [52]:
# add clustering labels
try:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except:
    pass

toronto_merged = df_toronto_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels']#.astype(int)
toronto_merged['Cluster Labels'].dropna(inplace=True)
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"[M1B] Rouge, Malvern",43.806686,-79.194353,5.0,Print Shop,Fast Food Restaurant,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
1,M1C,Scarborough,"[M1C] Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4.0,Bar,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,M1E,Scarborough,"[M1E] Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Electronics Store,Spa,Rental Car Location,Mexican Restaurant,Medical Center,Breakfast Spot,Pizza Place,Intersection,Donut Shop,Doner Restaurant
3,M1G,Scarborough,[M1G] Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
4,M1H,Scarborough,[M1H] Cedarbrae,43.773136,-79.239476,2.0,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Thai Restaurant,Bakery,Bank,Athletics & Sports,Department Store,Dessert Shop,Dim Sum Restaurant


In [53]:
toronto_merged['Cluster Labels'].value_counts()

2.0    81
0.0    13
3.0     2
5.0     2
1.0     1
4.0     1
Name: Cluster Labels, dtype: int64

In [54]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Conclusion: Postal codes in the same borough were not clustered together, it is possible that the each borough has an even amount of different venues arounds its area so a k-means clustering like this would give us a lot of 'similar' postal codes within a single cluster. ex. code ABC is in Borough X and code DEF is in Borough Y, both have many shopping areas and similar venues that cater towards the upper-middle class of the Borough, thus they are similar and would be clustered together